Project Starts By Implementing Yfinacne in this


In [1]:
import yfinance as yf
import pandas as pd

# Download last 5 years of Tesla stock data
data = yf.download("TSLA", start="2020-01-01", end="2025-01-01")

print(data.head())

C:\Users\Ansh Shokeen\AppData\Local\Temp\ipykernel_16516\1809090920.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("TSLA", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume
Ticker           TSLA       TSLA       TSLA       TSLA       TSLA
Date                                                             
2020-01-02  28.684000  28.713333  28.114000  28.299999  142981500
2020-01-03  29.534000  30.266666  29.128000  29.366667  266677500
2020-01-06  30.102667  30.104000  29.333332  29.364668  151995000
2020-01-07  31.270666  31.441999  30.224001  30.760000  268231500
2020-01-08  32.809334  33.232666  31.215334  31.580000  467164500


Pre Processing the data

In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Use only 'Close' price
close_data = data['Close'].values.reshape(-1,1)

# Normalize between 0 and 1
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(close_data)

# Prepare training dataset
X_train, y_train = [], []
window_size = 60   # use last 60 days to predict next day

for i in range(window_size, len(scaled_data)):
    X_train.append(scaled_data[i-window_size:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape for LSTM [samples, time_steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


Model Implimentation


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))  # prediction

# Compile
model.compile(optimizer='adam', loss='mean_squared_error')

# Train
model.fit(X_train, y_train, batch_size=32, epochs=20)


Epoch 1/20


C:\Users\Ansh Shokeen\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0252
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0055
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0044
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0035
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0031
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0030
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0027
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0026
Epoch 9/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0024
Epoch 10/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0023
Epoch 11/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0022
Epoch 12/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0023
Epoch 13/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0021
Epoch 14/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0021
Epoch 15/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0019
Epoch 16/20
38/

Making Predicitons

In [4]:
# Prepare test data
test_data = scaled_data[-(window_size+30):]  # last 60+30 days
X_test = []
for i in range(window_size, len(test_data)):
    X_test.append(test_data[i-window_size:i, 0])

X_test = np.array(X_test).reshape(-1, window_size, 1)

# Predictions
predicted_prices = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

print(predicted_prices[:10])  # first 10 predicted prices


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
[[316.0435 ]
 [318.59808]
 [321.87247]
 [324.36868]
 [325.6388 ]
 [327.77103]
 [327.95367]
 [326.9465 ]
 [324.58435]
 [323.47522]]


Real Time Predictions

In [5]:
import datetime
import yfinance as yf

today = datetime.date.today()
# Correct Tesla ticker = TSLA
live_data = yf.download("TSLA", start="2025-09-01", end=today)

if not live_data.empty:
    latest_price = live_data['Close'].iloc[-1]   # safer than [-1]
    print("Latest Price:", latest_price)
else:
    print("No data returned! Check ticker or date range.")



C:\Users\Ansh Shokeen\AppData\Local\Temp\ipykernel_16516\144089042.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  live_data = yf.download("TSLA", start="2025-09-01", end=today)
[*********************100%***********************]  1 of 1 completed

Latest Price: Ticker
TSLA    429.829987
Name: 2025-10-03 00:00:00, dtype: float64


Saving the model

In [6]:
model.save("stock_lstm.h5")

In [11]:
today = datetime.date.today()
start_date = today - datetime.timedelta(days=90)
live_data = yf.download("TSLA", start=start_date, end=today)

# Get the close prices
close_prices = live_data['Close'].values.reshape(-1, 1)

# Scale the data using the same scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_live_data = scaler.fit_transform(close_prices)

# Prepare the last 60 days for prediction
window_size = 60
last_60_days = scaled_live_data[-window_size:]
X_input = last_60_days.reshape(1, window_size, 1)

# Predict next day's price
predicted_scaled = model.predict(X_input)
predicted_price = scaler.inverse_transform(predicted_scaled)[0][0]

# Get current price (last actual price)
current_price = close_prices[-1][0]

# Calculate the change
price_change = predicted_price - current_price
percent_change = (price_change / current_price) * 100

# Display results
print("=" * 60)
print("TESLA STOCK PREDICTION")
print("=" * 60)
print(f"Current Price (Latest): ${current_price:.2f}")
print(f"Predicted Next Price:   ${predicted_price:.2f}")
print("-" * 60)
print(f"Expected Change:        ${price_change:+.2f}")
print(f"Expected % Change:      {percent_change:+.2f}%")
print("=" * 60)

if price_change > 0:
    print(f"📈 BULLISH: Stock is predicted to INCREASE by ${abs(price_change):.2f}")
elif price_change < 0:
    print(f"📉 BEARISH: Stock is predicted to DECREASE by ${abs(price_change):.2f}")
else:
    print(f"➡️  NEUTRAL: Stock is predicted to remain relatively STABLE")
print("=" * 60)

C:\Users\Ansh Shokeen\AppData\Local\Temp\ipykernel_16516\1340892462.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  live_data = yf.download("TSLA", start=start_date, end=today)
[*********************100%***********************]  1 of 1 completed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
TESLA STOCK PREDICTION
Current Price (Latest): $429.83
Predicted Next Price:   $433.67
------------------------------------------------------------
Expected Change:        $+3.84
Expected % Change:      +0.89%
📈 BULLISH: Stock is predicted to INCREASE by $3.84
